<a href="https://colab.research.google.com/github/shortcipher3/stackoverflow/blob/master/mediapipe_iris_2d_landmarks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/google/mediapipe/raw/master/mediapipe/models/iris_landmark.tflite

--2021-01-28 15:36:36--  https://github.com/google/mediapipe/raw/master/mediapipe/models/iris_landmark.tflite
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/google/mediapipe/master/mediapipe/models/iris_landmark.tflite [following]
--2021-01-28 15:36:37--  https://raw.githubusercontent.com/google/mediapipe/master/mediapipe/models/iris_landmark.tflite
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2640568 (2.5M) [application/octet-stream]
Saving to: ‘iris_landmark.tflite’

iris_landmark.tflit 100%[===================>]   2.52M  --.-KB/s    in 0.1s    

2021-01-28 15:36:37 (20.8 MB/s) - ‘iris_land

In [2]:
import tensorflow as tf
import numpy as np

In [4]:
img = np.random.randint(0, 256, (1,64,64,3), np.uint8)

img = (np.float32(img) - 0.0) / 255.0 # normalization (specified in tflite_converter_calculator, not in model card)

In [5]:
model_file = 'iris_landmark.tflite'
interpreter = tf.lite.Interpreter(
    model_path=model_file)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(f'input_details {input_details}')
print(f'output_details {output_details}')

# check the type of the input tensor
floating_model = input_details[0]['dtype'] == np.float32
print(f'is floating model: {floating_model}')

# NxHxWxC, H:1, W:2
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

interpreter.set_tensor(input_details[0]['index'], img)

interpreter.invoke()

output_eyes = interpreter.get_tensor(output_details[0]['index'])[0]
output_iris = interpreter.get_tensor(output_details[1]['index'])[0]

input_details [{'name': 'input_1', 'index': 0, 'shape': array([ 1, 64, 64,  3], dtype=int32), 'shape_signature': array([ 1, 64, 64,  3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
output_details [{'name': 'output_eyes_contours_and_brows', 'index': 384, 'shape': array([  1, 213], dtype=int32), 'shape_signature': array([  1, 213], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'output_iris', 'index': 385, 'shape': array([ 1, 15], dtype=int32), 'shape_signature': array([ 1, 15], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32)

In [6]:
print(f'output iris landmarks x {output_iris[::3]/64.0}') # tflite_tensors_to_landmarks_calculator normalization
print(f'output iris landmarks y {output_iris[1::3]/64.0}')
print(f'output iris landmarks z {output_iris[2::3]}')

print(f'output eye landmarks x {output_eyes[::3]/64.0}')
print(f'output eye landmarks y {output_eyes[1::3]/64.0}')
print(f'output eye landmarks z {output_eyes[2::3]}')

output iris landmarks x [0.55897963 0.6063238  0.56635237 0.5110681  0.5521186 ]
output iris landmarks y [0.56208456 0.5643482  0.51276773 0.56113124 0.6122207 ]
output iris landmarks z [-9.049727 -9.19979  -9.209205 -9.078963 -9.17062 ]
output eye landmarks x [0.35117516 0.38272172 0.41699478 0.4609751  0.523182   0.5803764
 0.6270789  0.6559522  0.66901934 0.3676813  0.38844615 0.42696217
 0.48754454 0.5484775  0.6088051  0.65074354 0.32605672 0.35947344
 0.40124327 0.46354222 0.5286034  0.59509146 0.64592785 0.6756573
 0.6925186  0.31955034 0.34157807 0.38402987 0.45450583 0.53435564
 0.6046024  0.6705826  0.30013934 0.32728282 0.36449912 0.43831757
 0.52824885 0.6109411  0.6694243  0.7064492  0.73264164 0.29185337
 0.3043334  0.3538466  0.43054134 0.5261933  0.64126605 0.7090034
 0.25805146 0.24881075 0.25412142 0.30552655 0.39131805 0.51375514
 0.2093114  0.27279893 0.31911027 0.40632808 0.5311821  0.6195576
 0.6850815  0.72896266 0.7576413  0.2048842  0.21858425 0.2712011
 0.3647